# Coördinated Spiking Model ($\textit{N}$ neurons)

Install required packages:

In [1]:
import numpy as np
import pylab as pl

Network size:

In [29]:
N = 3

Initialize parameters:

In [36]:
# timesteps
dt = 0.01
iterations = 500
xtime = np.arange(0, iterations) * dt

# read-out over time
xhat = np.ones(iterations) * 0

# signal over time
x = 5
x_t = np.ones(iterations) * x

step = 75
x_t[0:step] = 0
x_t[iterations - 2 * step:iterations] = 3

# derivative of signal over time
dxdt = np.diff(x_t)/dt

# neural input over time
c = x_t[:-1] + dxdt

# voltage/error over time
V = np.zeros((N, iterations))
dVdt = np.zeros((N, iterations))
voltages = np.zeros((N, iterations))

# spikes over time
spike = np.zeros((N, iterations))
o = np.zeros((N, iterations))

# read-out weight
gamma = 2

# threshold
T = gamma**2 / 2

Run simulation:

In [37]:
for t in range(1, iterations):
    
    # update read-out
    dxhatdt = - xhat[t - 1] + gamma * np.sum(o[:, t - 1])
    xhat[t] = xhat[t - 1] + dt * dxhatdt
    
    # compute error
    # V[:, t] = gamma * (x_t[t] - xhat[t])
    
    dVdt = -voltages[:, t - 1] + gamma * c[t - 1] - np.sum(o[:, t - 1]) * (gamma ** 2)
    voltages[:, t] =  voltages[:, t - 1] + dt * dVdt
    above_threshold = np.where(voltages[:, t] > T)
    
    # reset neuron when threshold is exceeded
    if len(above_threshold[0]) > 0:
        
        # select random neuron who's voltage is above threshold
        spike_neuron = np.random.randint(0, len(above_threshold[0]))
        
        # update spikes
        o[spike_neuron][t] = 1/dt
        spike[spike_neuron][t] = 1

Plot results:

In [38]:
plots = (2 * N) + 1
pl.subplots(plots, 1, figsize=(5,6))

# enable LaTeX
pl.rc('text', usetex=True)

# plot input and read-out
pl.subplot(plots, 1, 1)
pl.plot(xhat, color='blue', label=r'$\hat{x}$')
pl.plot(x_t, color='red', label=r'$x$')
pl.yticks([])
pl.xticks([])
# pl.legend()

counter = 2

for i in range(N):
    
    ylabel = r'$V_{0}$'.format(str(i + 1))
    
    # plot voltage
    pl.subplot(plots, 1, counter)
    pl.plot(voltages[i], color='black')
    # pl.ylabel(ylabel)
    pl.yticks([])
    pl.xticks([])
    
    counter = counter + 1

    # plot spikes
    pl.subplot(plots, 1, counter)
    pl.plot(spike[i], color='lightgrey')
    pl.yticks([])
    pl.xticks([])
    
    counter = counter + 1

# pl.xlabel('time')